## Dependencies

In [1]:
import glob
import warnings
from tensorflow_hub import KerasLayer
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling1D, SpatialDropout1D, Concatenate
from googleqa_utilityscript import *
from googleqa_map_utilityscript import *
import bert_tokenization as tokenization


SEED = 0
seed_everything(SEED)
warnings.filterwarnings("ignore")

## Load data

In [2]:
BERT_PATH = '/kaggle/input/tf-hub-bert-base/bert_base_uncased'
VOCAB_PATH = BERT_PATH + '/assets/vocab.txt'
model_question_path_list = glob.glob('/kaggle/input/129-googleq-a-train-3fold-bert-base-unc-question/' + '*question.h5')
model_answer_path_list = glob.glob('/kaggle/input/129-googleq-a-train-3fold-bert-base-unc-answer/' + '*answer.h5')
model_question_path_list.sort()
model_answer_path_list.sort()
print('Models to predict:', model_question_path_list + model_answer_path_list)

test = pd.read_csv('/kaggle/input/google-quest-challenge/test.csv')

print('Test samples: %s' % len(test))
display(test.head())

Models to predict: ['/kaggle/input/129-googleq-a-train-3fold-bert-base-unc-question/model_fold_1_question.h5', '/kaggle/input/129-googleq-a-train-3fold-bert-base-unc-question/model_fold_2_question.h5', '/kaggle/input/129-googleq-a-train-3fold-bert-base-unc-question/model_fold_3_question.h5', '/kaggle/input/129-googleq-a-train-3fold-bert-base-unc-answer/model_fold_1_answer.h5', '/kaggle/input/129-googleq-a-train-3fold-bert-base-unc-answer/model_fold_2_answer.h5', '/kaggle/input/129-googleq-a-train-3fold-bert-base-unc-answer/model_fold_3_answer.h5']
Test samples: 476


,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host
0,39,Will leaving corpses lying around upset my pri...,I see questions/information online about how t...,Dylan,https://gaming.stackexchange.com/users/64471,There is no consequence for leaving corpses an...,Nelson868,https://gaming.stackexchange.com/users/97324,http://gaming.stackexchange.com/questions/1979...,CULTURE,gaming.stackexchange.com
1,46,Url link to feature image in the portfolio,I am new to Wordpress. i have issue with Featu...,Anu,https://wordpress.stackexchange.com/users/72927,I think it is possible with custom fields.\n\n...,Irina,https://wordpress.stackexchange.com/users/27233,http://wordpress.stackexchange.com/questions/1...,TECHNOLOGY,wordpress.stackexchange.com
2,70,"Is accuracy, recoil or bullet spread affected ...","To experiment I started a bot game, toggled in...",Konsta,https://gaming.stackexchange.com/users/37545,You do not have armour in the screenshots. Thi...,Damon Smithies,https://gaming.stackexchange.com/users/70641,http://gaming.stackexchange.com/questions/2154...,CULTURE,gaming.stackexchange.com
3,132,Suddenly got an I/O error from my external HDD,I have used my Raspberry Pi as a torrent-serve...,robbannn,https://raspberrypi.stackexchange.com/users/17341,Your Western Digital hard drive is disappearin...,HeatfanJohn,https://raspberrypi.stackexchange.com/users/1311,http://raspberrypi.stackexchange.com/questions...,TECHNOLOGY,raspberrypi.stackexchange.com
4,200,Passenger Name - Flight Booking Passenger only...,I have bought Delhi-London return flights for ...,Amit,https://travel.stackexchange.com/users/29089,I called two persons who work for Saudia (tick...,Nean Der Thal,https://travel.stackexchange.com/users/10051,http://travel.stackexchange.com/questions/4704...,CULTURE,travel.stackexchange.com


In [3]:
question_target_cols = ['question_asker_intent_understanding','question_body_critical', 'question_conversational', 
                        'question_expect_short_answer', 'question_fact_seeking', 'question_has_commonly_accepted_answer',
                        'question_interestingness_others', 'question_interestingness_self', 'question_multi_intent', 
                        'question_not_really_a_question', 'question_opinion_seeking', 'question_type_choice',
                        'question_type_compare', 'question_type_consequence', 'question_type_definition', 
                        'question_type_entity', 'question_type_instructions', 'question_type_procedure',
                        'question_type_reason_explanation', 'question_type_spelling', 'question_well_written']
answer_target_cols = ['answer_helpful', 'answer_level_of_information', 'answer_plausible', 'answer_relevance',
                      'answer_satisfaction', 'answer_type_instructions', 'answer_type_procedure', 
                      'answer_type_reason_explanation', 'answer_well_written']
target_cols = question_target_cols + answer_target_cols

## Pre-process data

In [4]:
text_features = ['question_title', 'question_body', 'answer']
text_features_question = ['question_title', 'question_body']
text_features_answer = ['question_title', 'answer']

# for feature in text_features:
#     # Lower
#     test[feature] = test[feature].apply(lambda x: x.lower())
#     # Map misspellings
#     test[feature] = test[feature].apply(lambda x: map_misspellings(x))
#     # Map contractions
#     test[feature] = test[feature].apply(lambda x: map_contraction(x))
#     # Trim text
#     test[feature] = test[feature].apply(lambda x: x.strip())

# Model parameters

In [5]:
N_CLASS = len(target_cols)
N_CLASS_QUESTION = len(question_target_cols)
N_CLASS_ANSWER = len(answer_target_cols)
MAX_SEQUENCE_LENGTH = 512
MAX_LENGTH_TITLE = 30
MAX_LENGTH_QUESTION = 479
MAX_LENGTH_ANSWER = 479

## Test set

In [6]:
tokenizer = tokenization.FullTokenizer(VOCAB_PATH, do_lower_case=True)

# Test features
X_test_question = compute_input_arays_2(test, text_features_question, tokenizer, MAX_SEQUENCE_LENGTH, 
                                        MAX_LENGTH_TITLE, MAX_LENGTH_QUESTION, ignore_first_sep=False)
X_test_answer = compute_input_arays_2(test, text_features_answer, tokenizer, MAX_SEQUENCE_LENGTH, 
                                      MAX_LENGTH_TITLE, MAX_LENGTH_ANSWER, ignore_first_sep=False)

# Model

In [7]:
def question_model_fn():
    input_word_ids = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_word_ids')
    input_masks = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_masks')
    segment_ids = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='segment_ids')

    bert_layer = KerasLayer(BERT_PATH, trainable=True)
    pooled_output, sequence_output = bert_layer([input_word_ids, input_masks, segment_ids])

    x = GlobalAveragePooling1D()(sequence_output)
    x = Dropout(0.2)(x)
    output = Dense(N_CLASS_QUESTION, activation="sigmoid", name="output")(x)

    model = Model(inputs=[input_word_ids, input_masks, segment_ids], outputs=output)
    
    return model

def answer_model_fn():
    input_word_ids = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_word_ids')
    input_masks = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_masks')
    segment_ids = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='segment_ids')

    bert_layer = KerasLayer(BERT_PATH, trainable=True)
    pooled_output, sequence_output = bert_layer([input_word_ids, input_masks, segment_ids])

    x = GlobalAveragePooling1D()(sequence_output)
    x = Dropout(0.2)(x)
    output = Dense(N_CLASS_ANSWER, activation="sigmoid", name="output")(x)

    model = Model(inputs=[input_word_ids, input_masks, segment_ids], outputs=output)
    
    return model

# Make predictions

In [8]:
Y_test = np.zeros((len(test), N_CLASS))

for index, model_path in enumerate(model_question_path_list):
    model_question = question_model_fn()
    model_answer = answer_model_fn()
    model_question.load_weights(model_path)
    model_answer.load_weights(model_answer_path_list[index])
    
    question_preds = model_question.predict(X_test_question)
    answer_preds = model_answer.predict(X_test_answer)
    Y_test += np.hstack([question_preds, answer_preds]) / len(model_question_path_list)

In [9]:
submission = pd.read_csv('/kaggle/input/google-quest-challenge/sample_submission.csv')
submission[target_cols] = Y_test
submission.to_csv("submission.csv", index=False)
display(submission.head())
display(submission.describe())

,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,39,0.951789,0.642501,0.145374,0.552619,0.626644,0.513667,0.680546,0.647969,0.667960,...,0.907906,0.920962,0.598019,0.955646,0.958553,0.836552,0.034541,0.041871,0.756018,0.905735
1,46,0.856371,0.510560,0.007229,0.765291,0.744317,0.917115,0.536344,0.415269,0.107336,...,0.603531,0.934679,0.640079,0.970106,0.977529,0.859968,0.915557,0.084280,0.062422,0.871902
2,70,0.911469,0.627831,0.021246,0.754217,0.865304,0.911201,0.599192,0.492041,0.159511,...,0.859754,0.951187,0.593204,0.974524,0.974298,0.866615,0.046379,0.039408,0.904281,0.886682
3,132,0.859721,0.405088,0.006539,0.705915,0.742178,0.908680,0.532316,0.379617,0.163885,...,0.651063,0.954837,0.681353,0.977383,0.983961,0.894735,0.792617,0.139775,0.633989,0.885868
4,200,0.910195,0.394671,0.031981,0.812011,0.782059,0.890113,0.653162,0.582283,0.091656,...,0.601431,0.914387,0.648605,0.959910,0.958611,0.826939,0.266505,0.111168,0.553889,0.882703


,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
count,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,...,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000
mean,5029.186975,0.890405,0.565586,0.031211,0.725839,0.786973,0.837631,0.571430,0.468323,0.241129,...,0.776986,0.933981,0.640445,0.966150,0.971703,0.863382,0.528199,0.117913,0.510738,0.897537
std,2812.670060,0.043678,0.132950,0.048018,0.088365,0.090800,0.120155,0.055614,0.093750,0.195341,...,0.092695,0.027299,0.045582,0.011376,0.013752,0.040967,0.317928,0.048597,0.281424,0.025773
min,39.000000,0.763168,0.335598,0.004161,0.272544,0.464106,0.340286,0.468610,0.331022,0.016699,...,0.572205,0.798433,0.537134,0.916742,0.907408,0.697504,0.005702,0.012501,0.043811,0.802858
25%,2572.000000,0.858685,0.456312,0.007781,0.682856,0.742967,0.817090,0.527389,0.397697,0.089917,...,0.701054,0.919718,0.611796,0.960969,0.965628,0.842718,0.203363,0.084809,0.267443,0.881929
50%,5093.000000,0.892919,0.553194,0.012156,0.729513,0.784487,0.883369,0.561716,0.437188,0.164935,...,0.775467,0.941048,0.632988,0.968159,0.975679,0.869137,0.627338,0.117326,0.494098,0.900536
75%,7482.000000,0.926128,0.678702,0.029103,0.777202,0.846678,0.912133,0.610926,0.516038,0.368313,...,0.860760,0.953747,0.663602,0.974436,0.981624,0.892991,0.825886,0.147685,0.777645,0.916380
max,9640.000000,0.970907,0.878766,0.350675,0.935801,0.967105,0.961934,0.725710,0.783331,0.847930,...,0.937561,0.974226,0.791171,0.984967,0.988731,0.939168,0.940424,0.288593,0.983275,0.942151
